In [1]:
import os

In [2]:
os.environ['SPARK_HOME'] = "/usr/local/spark-3.5.0-bin-hadoop3"
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-17-openjdk-amd64/"

In [3]:
import findspark
from pyspark.sql import *
from pyspark import SparkConf
import pynessie

In [4]:
pynessie_version = pynessie.__version__

In [5]:
NESSIE_URI = os.environ.get("NESSIE_URI") ## Nessie Server URI
WAREHOUSE = os.environ.get("WAREHOUSE") ## BUCKET TO WRITE DATA TOO
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY") ## AWS CREDENTIALS
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_KEY") ## AWS CREDENTIALS
AWS_S3_ENDPOINT= os.environ.get("AWS_S3_ENDPOINT") ## MINIO ENDPOINT

In [11]:
conf = SparkConf()
# we need iceberg libraries and the nessie sql extensions
conf.set(
    "spark.jars.packages",
    f"org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1,org.projectnessie:nessie-spark-3.2-extensions:{pynessie_version}",
)
# ensure python <-> java interactions are w/ pyarrow
conf.set("spark.sql.execution.pyarrow.enabled", "true")
# create catalog dev_catalog as an iceberg catalog
conf.set("spark.sql.catalog.dev_catalog", "org.apache.iceberg.spark.SparkCatalog")
# tell the dev_catalog that its a Nessie catalog
conf.set("spark.sql.catalog.dev_catalog.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
# set the location for Nessie catalog to store data. Spark writes to this directory
conf.set("spark.sql.catalog.dev_catalog.warehouse", WAREHOUSE)
# set the location of the nessie server. In this demo its running locally. There are many ways to run it (see https://projectnessie.org/try/)
conf.set("spark.sql.catalog.dev_catalog.uri", NESSIE_URI)
# default branch for Nessie catalog to work on
conf.set("spark.sql.catalog.dev_catalog.ref", "main")
# use no authorization. Options are NONE AWS BASIC and aws implies running Nessie on a lambda
conf.set("spark.sql.catalog.dev_catalog.auth_type", "NONE")
# enable the extensions for both Nessie and Iceberg
conf.set(
    "spark.sql.extensions",
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSpark32SessionExtensions",
)

In [12]:
spark = SparkSession.builder.getOrCreate()
print("Spark Running")

Spark Running


In [13]:
spark.sql("CREATE BRANCH dev IN dev_catalog FROM main")

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'BRANCH'.(line 1, pos 7)

== SQL ==
CREATE BRANCH dev IN dev_catalog FROM main
-------^^^
